In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import  Input,Conv2D, MaxPooling2D
from keras.layers import Activation, Dense,Rescaling, MultiHeadAttention,BatchNormalization, Reshape
from tensorflow.keras import activations
from keras.regularizers import l2
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten,Input, BatchNormalization, GlobalMaxPool1D,LSTM
from keras.models import Model
from tensorflow.keras.losses import sparse_categorical_crossentropy, categorical_crossentropy
import tensorflow_addons as tfa
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter
import os
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle
from scipy.signal import savgol_filter
from scipy.interpolate import CubicSpline
from sklearn import preprocessing
import scipy.spatial as sp
from sklearn.decomposition import PCA
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
from scipy.stats import skew, kurtosis, iqr
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import r2_score, mean_squared_error
from numpy.linalg import norm
from scipy import interpolate
import librosa
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import Sequence
import librosa.display
import gc
from ssqueezepy import cwt
from ssqueezepy.visuals import plot, imshow
import cv2
import re

In [2]:
#get list of all file names in directory "data"
file_names = os.listdir(r"D:/Research_work/Trace_Analysis/data5")

#get the part of names in file_names before '-'
input_variables = [name.split('-')[0] for name in file_names]

column_names =['ignore', 'command (read/write)', 'physical address', 'size','gem5 flags', 'timestamp','program_counter']
#find unique elements in input_variables
unique_input_variables = np.unique(input_variables)
print("All Input Variables (Classes): ")
print(unique_input_variables)
print("---------------------")


master_dataset_x=[]
master_dataset_y=[]


knowledge=[]
knowledge=[]

for names in file_names:
    #check if names contain string 'icachetrace'
    if 'dcachetrace' in names:
        number_in_txt=re.findall(r'\d+', names)
        number_in_txt=number_in_txt[0]
        input_variable=names.split('-')[0]
        df = pd.read_csv("D:/Research_work/Trace_Analysis/data5/"+ names, sep=',', header=None)
        df.columns = column_names

        #find unique elements in column 'command (read/write)'
        unique_commands=df['command (read/write)'].unique()

        #create dictionary of unique_commands that integer encode the column 'command (read/write)'
        dict_commands = {unique_commands[i]:i for i in range(len(unique_commands))}

        #integer encode 'comand (read/write)' column
        df['command (read/write)'] = df['command (read/write)'].map(dict_commands)

        # convert df to np arrays
        df = df.to_numpy()
        possible_length=len(df)
        master_dataset_x.append(df)
        master_dataset_y.append(input_variable)
        knowledge.append(input_variable+number_in_txt+'D')

    if 'icachetrace' in names:
        number_in_txt=re.findall(r'\d+', names)
        number_in_txt=number_in_txt[0]
        input_variable=names.split('-')[0]
        df = pd.read_csv("D:/Research_work/Trace_Analysis/data5/"+ names, sep=',', header=None)
        df.columns = column_names

        #find unique elements in column 'command (read/write)'
        unique_commands=df['command (read/write)'].unique()

        #create dictionary of unique_commands that integer encode the column 'command (read/write)'
        dict_commands = {unique_commands[i]:i for i in range(len(unique_commands))}

        #integer encode 'comand (read/write)' column
        df['command (read/write)'] = df['command (read/write)'].map(dict_commands)

        # convert df to np arrays
        df = df.to_numpy()
        possible_length=len(df)
        master_dataset_x.append(df)
        master_dataset_y.append(input_variable)
        knowledge.append(input_variable+number_in_txt+'I')






print('size of master_dataset_x: ',len(master_dataset_x))
print('dimensions of features in master_dataset_x: ',master_dataset_x[0].shape)


#getting inhomogenous lengths
len_seq=[]
for i in range(len(master_dataset_x)):
    len_seq.append(master_dataset_x[i].shape[0])

min_len_seq=min(len_seq)
print('minimum length of sequence: ',min_len_seq)
max_len_seq=max(len_seq)
print('maximum length of sequence: ',max_len_seq)



# #padding zeros
threshold=max_len_seq
for i in range(len(master_dataset_x)):
    #pad zeros if len of sequence is less than threshold
    if master_dataset_x[i].shape[0]<threshold:
        master_dataset_x[i]=np.pad(master_dataset_x[i],((0,threshold-master_dataset_x[i].shape[0]),(0,0)),'constant')


#cropping data
threshold=700
for i in range(len(master_dataset_x)):
    master_dataset_x[i]=master_dataset_x[i][:threshold,:]



data_X_list=master_dataset_x
data_Y_list=master_dataset_y


#convert to np array
data_X=np.array(data_X_list)
data_Y=np.array(data_Y_list)



print(data_X.shape)

#horizonatlly stacking enteries of data_X corresponding to d and i in 'knowledge'
data_X_new=[]
data_Y_new=[]
count=0
for i in range(len(knowledge)):
    k_name=knowledge[i]
    if 'D' in k_name:
        considered_name_d=k_name[:k_name.index('D')]
        for j in range(len(knowledge)):
            k_name2=knowledge[j]
            if 'I' in k_name2:
                considered_name_i=k_name2[:k_name2.index('I')]
                if considered_name_d==considered_name_i:
                    #get number in considered_name_d
                    number_in_txt=re.findall(r'\d+', considered_name_d)
                    number_in_txt=number_in_txt[0]
                    #get string before number in considered_name_d
                    probable_class=considered_name_d[:considered_name_d.index(number_in_txt)]
                    data_Y_new.append(probable_class)
                    data_X_new.append(np.hstack((data_X[i],data_X[j])))
                    break




data_X_new=np.array(data_X_new)   
data_X=data_X_new.copy()  
data_Y=np.array(data_Y_new)


#shuffle
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

data_X, data_Y = unison_shuffled_copies(data_X, data_Y)



#------------------------------------------------------------------------------------------


Omega_Y_Out=data_Y.copy()

# correcting labels
for i in range(len(data_Y)):
    subject=data_Y[i]
    #check if subject has 10 characters
    if len(subject)<=4:
        data_Y[i]='class1'
    if len(subject)<=6 and len(subject)>4:
        data_Y[i]='class2'
    if len(subject)<=10 and len(subject)>6:
        data_Y[i]='class3'

Omega_Y_In=data_Y.copy()





#slicing some variables
vars_2_keep=[i for i in range(14)]
data_X=data_X[:,:,vars_2_keep]
num_vars=len(vars_2_keep)




# #####-------smoothing-----##########
for i in range(len(data_X)):
    for j in range(data_X.shape[2]):
        sig=data_X[i,:,j]
        check_sig=sig.copy()
        smooth_sig=gaussian_filter(sig, sigma=2)
        data_X[i,:,j]=smooth_sig

# ################################


#partition data for now--again
part=len(data_X)
data_X=data_X[:part]
data_Y=data_Y[:part]




print("Unique Labels: ",np.unique(data_Y))
print("data_X shape: ", data_X.shape)
print("data_Y shape: ", data_Y.shape)





#one hot encode data_Y
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data_Y)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
num_classes=int(np.max(integer_encoded)+1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded).astype(float)

print("one_hot_encoded shape: ", onehot_encoded.shape)
mapping = dict(zip(label_encoder.classes_, onehot_encoder.transform(label_encoder.transform(label_encoder.classes_).reshape(num_classes, 1))))
for key,value in mapping.items():
    print("",key," -----------------------------> ",value)



#--------------------------------------------------------------------------------------------------------------

#--------------------------------------------------------------------------------------------------------------



#PCA thingy from my COVID paper

data_X_copy=data_X.copy()
# -----------------sklearn minmax scaler
for i in range(data_X.shape[2]):
    scaler = StandardScaler()
    stranded_data=data_X_copy[:,:,i]
    standardized_stranded_data = scaler.fit_transform(stranded_data)
    data_X[:,:,i]=standardized_stranded_data







#----------------------sine transformation
amp1=1.0
amp2=0.01
for i in range(len(data_X)):
    for j in range(data_X.shape[2]):
        if np.max(data_X[i,:,j])!=0:
            data_X[i,:,j]=data_X[i,:,j]+amp1*np.sin(2*np.pi*data_X[i,:,j]/np.max(data_X[i,:,j]))+amp2*np.random.uniform(low=0.5, high=1.3, size=(data_X.shape[1],))
        else:
            data_X[i,:,j]=data_X[i,:,j]+amp1*np.sin(2*np.pi*data_X[i,:,j])+amp2*np.random.uniform(low=0.5, high=1.3, size=(data_X.shape[1],))




# wavelet spectrogram generation
import matplotlib
matplotlib.use('Agg')

IMG_SIZE=256


for i in range(len(data_X)):
    Im=[]
    for j in range(data_X.shape[2]):
        sig=data_X[i,:,j]
        fig, ax = plt.subplots()
        Wx, scales = cwt(sig, 'morlet')
        imshow(Wx, abs=1)
        fig.savefig('images_combined\im'+str(i)+'_'+str(j)+'.jpg', dpi=300, bbox_inches='tight', pad_inches=0)
        gc.collect()


    #print progress on same line
    print("Progress: ", i, "/", len(data_X), end="\r")


np.save('data_X_combined.npy',data_X)
np.save('data_Y_combined.npy',data_Y)
np.save('onehot_encoded_combined.npy',onehot_encoded)




All Input Variables (Classes): 
['air' 'art' 'government' 'health']
---------------------
size of master_dataset_x:  24
dimensions of features in master_dataset_x:  (675, 7)
minimum length of sequence:  192
maximum length of sequence:  2560
(24, 700, 7)
Unique Labels:  ['class1' 'class2' 'class3']
data_X shape:  (12, 700, 14)
data_Y shape:  (12,)
one_hot_encoded shape:  (12, 3)
 class1  ----------------------------->  [1. 0. 0.]
 class2  ----------------------------->  [0. 1. 0.]
 class3  ----------------------------->  [0. 0. 1.]
